# Interpolation

Import what we need from narcpack and other modules:

In [ ]:
from narcpack.interp import Rbf,Polyinterp
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Radial basis functions (and polynomial)

These are the basis functions available to the `Rbf` class (along with some colors):

In [ ]:
functions = ['multiquadric','inverse multiquadric','inverse quadratic','gaussian','linear','cubic','quintic']
ls = ['tab:blue','tab:orange','tab:red','tab:brown','tab:pink','tab:gray','tab:cyan']

Now we'll define a function to help us make these plots. It takes a function `g` to interpolate using `n` points spaced over `interval` (with linear spacing or random spacing if `rand` is true).

In [ ]:
def make_Rbf_plots(g, n=10, interval=[-1,1], rand=False):
    fig, ax = plt.subplots(1,2,figsize=[12,4])
    if rand:
        x = np.sort(np.random.uniform(interval[0],interval[1],n))
    else:
        x = np.linspace(interval[0],interval[1],n)
    xfine = np.linspace(interval[0]-0.25*np.abs(interval[0]),interval[1]+0.25*np.abs(interval[1]),1000)
    ax[0].plot(xfine,g(xfine),'--k',label='True')
    yl = ax[0].get_ylim()
    p = Polyinterp(x, g(x))
    ax[0].plot(xfine,p(xfine),'k',label='Polynomial')
    ax[1].plot(xfine,np.abs(p(xfine)-g(xfine)),'k')
    for i, function in enumerate(functions):
        r = Rbf(x, g(x), function=function)
        ax[0].plot(xfine,r(xfine),ls[i],label='Rbf '+function)
        ax[0].plot(x, g(x), 'ok')
        ax[1].semilogy(xfine,np.abs(r(xfine)-g(xfine)),ls[i])
    if (ax[0].get_ylim()[0] < 2*yl[0]) | (ax[0].get_ylim()[1] > 2*yl[1]):
        ax[0].set_ylim([yl[0]-1,yl[1]+1])
    ax[1].set_ylim([np.max([1e-9,ax[1].get_ylim()[0]]),ax[1].get_ylim()[1]])
    fig.legend(loc=5,bbox_to_anchor=[1.2,0.5])
    fig.tight_layout()

Now we'll generate some plots.

In [ ]:
make_Rbf_plots(lambda x : np.abs(x), n=10, rand=False)

In [ ]:
make_Rbf_plots(lambda x : np.sign(x), n=10, rand=True)

In [ ]:
make_Rbf_plots(lambda x : 1.0/(1.0+25.0*x**2), n=50, rand=False)

In [ ]:
make_Rbf_plots(lambda x : np.exp(x), interval=[0,8], n=50, rand=False)

In [ ]:
make_Rbf_plots(lambda x : 4*np.sin(x)-2*np.cos(0.5*x)+np.sin(3*x), interval=[-10,10], n=100, rand=True)

## Timing

Now we'll try timing our methods. First we need the `time` module.

In [ ]:
import time

First we'll try a large problem. `Polyinterp` can't handle it.

In [ ]:
g = lambda x : 4*np.sin(x)-2*np.cos(0.5*x)+np.sin(3*x)
x = np.linspace(-10,10,1e3)
t = time.time()
p = Polyinterp(x, g(x))
elapsed = time.time()-t
print('Time\t\t\tAbsolute error\t\tMethod')
print('=====================================================================')
print(str(elapsed)+'\t'+str(np.max(np.abs(p(x)-g(x))))+'\t\t\tPolynomial')
for function in functions:
    t = time.time()
    r = Rbf(x, g(x), function=function)
    elapsed = time.time()-t
    print(str(elapsed)+'\t'+str(np.max(np.abs(r(x)-g(x))))+'\tRbf '+function)

Now let's take the minimum timing over a bunch of small problems.

In [ ]:
g = lambda x : 4*np.sin(x)-2*np.cos(0.5*x)+np.sin(3*x)
x = np.linspace(-4,4,20)
elapsed = np.Inf
for i in range(1000):
    t = time.time()
    p = Polyinterp(x, g(x))
    elapsed = np.min([elapsed,time.time()-t])
print('Time\t\t\tMethod')
print('=============================================')
print(str(elapsed)+'\tPolynomial')
for function in functions:
    elapsed = np.Inf
    for i in range(1000):
        t = time.time()
        r = Rbf(x, g(x), function=function)
        elapsed = np.min([elapsed,time.time()-t])
    print(str(elapsed)+'\tRbf '+function)